In [2]:
pip install evaluate transformers==4.53.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 42.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarr

In [4]:
pip install jiwe

ERROR: Could not find a version that satisfies the requirement jiwe (from versions: none)
ERROR: No matching distribution found for jiwe
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import requests
import torch
from PIL import Image
import json
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from torch.optim import AdamW
import transformers
import re
from transformers import AutoModelForCausalLM,AutoProcessor,get_scheduler

In [6]:
df = pd.read_excel("/kaggle/input/docsmetric/Судебный приказ/labels.xlsx")

In [7]:
def extract_and_clean_text(raw_str):
    match = re.search(r'"text"\s*:\s*"((?:[^"\\]|\\.)*?)"', raw_str)
    if not match:
        return ""
    text = match.group(1)
    text = text.replace('\\n', ' ').replace('\\t', ' ').replace('\\r', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
df["cleaned_text"] = df["response"].apply(extract_and_clean_text)

print(df[["cleaned_text"]])

                                         cleaned_text
0   Дело № СУДЕБНЫЙ ПРИКАЗ 03 июня 2008 года Миров...
1   АРБИТРАЖНЫЙ СУД СВЕРДЛОВСКОЙ ОБЛАСТИ 620075 г....
2   СУДЕБНЫЙ ПРИКАЗ №2-182/1/2009 г. Кольчугино 19...
3   СУДЕБНЫЙ ПРИКАЗ № 2-4803/2016 25 ноября 2016 г...
4   Судебный участок № 1 Бежицкого судебного район...
5   Копия Дело №2-403/2016 ОПРЕДЕЛЕНИЕ город Брянс...
6   СУДЕБНЫЙ ПРИКАЗ мая 2015 года г. Ростов-на-Дон...
7   Судебный участок № 4 Ленинского района г.Влади...
8   АРБИТРАЖНЫЙ СУД САРАТОВСКОЙ ОБЛАСТИ 410002, г....
9   СУДЕБНЫЙ ПРИКАЗ О судебном участке № 8 Ленинск...
10  Судебный участок № 117 района Гольяново г. Мос...
11  Копия Дело № СП2-7823/11 СУДЕБНЫЙ ПРИКАЗ «27» ...
12  СУДЕБНЫЙ ПРИКАЗ 19 января 2021 года г. Барнаул...
13  Судебный участок № 1 Ленинского района г. Екат...
14  СУДЕБНЫЙ ПРИКАЗ г. Красногорск Московской обла...
15  Судебный участок № 7 Беловского городского суд...
16  Дело № 2-667/2014 СУДЕБНЫЙ ПРИКАЗ 07 июля 2014...
17  Дело № [REDACTED] Именем

In [8]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
normalizer = BasicTextNormalizer()

In [9]:
df["cleaned_text"] = df["cleaned_text"].apply(normalizer)

In [17]:
df.head()

,image_name,prompt,response,model,edited,cleaned_text
0,001.jpg,На входе ты получишь изображение скана докумен...,"```json\n{\n ""text"": ""Дело №\n\nСУДЕБНЫЙ ПР...",claude-3-5-sonnet-20240620,0,дело no судебный приказ 03 июня 2008 года миро...
1,002.jpg,На входе ты получишь изображение скана докумен...,"```json\n{\n\t""text"": ""АРБИТРАЖНЫЙ СУД СВЕРДЛО...",gpt-4o-2024-05-13,0,арбитражный суд свердловской области 620075 г ...
2,003.jpg,На входе ты получишь изображение скана докумен...,"```json\n{\n ""text"": ""СУДЕБНЫЙ ПРИКАЗ №2-18...",claude-3-5-sonnet-20240620,0,судебный приказ no2 182 1 2009 г кольчугино 19...
3,004.jpg,На входе ты получишь изображение скана докумен...,"```json\n{\n ""text"": ""СУДЕБНЫЙ ПРИКАЗ № 2-4...",claude-3-5-sonnet-20240620,0,судебный приказ no 2 4803 2016 25 ноября 2016 ...
4,005.jpg,На входе ты получишь изображение скана докумен...,"```json\n{\n ""text"": ""Судебный участок № 1\...",claude-3-5-sonnet-20240620,0,судебный участок no 1 бежицкого судебного райо...


In [15]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
model_id = "microsoft/Florence-2-large-ft"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
).eval().to(device)


processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)


config.json: 0.00B [00:00, ?B/s]

configuration_florence2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value T

modeling_florence2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

processing_florence2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
model_path = "/kaggle/input/florence2-finetune/pytorch/default/1/modelFNEW_DOC_checkpoints_epoch4"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
).to(device)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True,)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [30]:
import torch, gc
model = None
processor = None
result_model = None
gc.collect()
torch.cuda.empty_cache()
print("GPU memory should be freed (but restart kernel to be sure)")

GPU memory should be freed (but restart kernel to be sure)


In [11]:
def run_example(image_path):
    image = Image.open(f"/kaggle/input/docsmetric/Судебный приказ/{image_path}").convert("RGB")
    prompt = '<OCR>'

    inputs = processor(text=prompt, images=image, return_tensors="pt")
    
    # ✅ Cast inputs to model's dtype and move to device
    inputs = {
        k: v.to(device, dtype=model.dtype if k == "pixel_values" else None)
        for k, v in inputs.items()
    }

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=prompt,
        image_size=(image.width, image.height)
    )
    return str(parsed_answer["<OCR>"])

In [16]:
df_test = df.iloc[:2

In [ ]:
df['florence2_ft'] = df["image_name"].apply(run_example)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [ ]:
df['florence2_ft_norm'] = df["florence2_ft"].apply(normalizer)

In [4]:
from evaluate import load

wer_metric = load("wer")

2025-10-16 17:08:36.241905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760634516.438479      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760634516.493243      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
wer = wer_metric.compute(
    references=df['cleaned_text'].tolist(),predictions=df['florence2_norm'].tolist()
)
wer

0.9462505750651741

In [29]:
cer_metric = load("cer")
cer = cer_metric.compute(references= df['cleaned_text'].tolist(), predictions= df['florence2_norm'].tolist())
print(cer)

0.7525382914433241


In [ ]:
wer = wer_metric.compute(
    references=df['cleaned_text'].tolist(),predictions=df['florence2_ft_norm'].tolist()
)
wer

In [ ]:
cer_metric = load("cer")
cer = cer_metric.compute(references= df['cleaned_text'].tolist(), predictions= df['florence2_ft_norm'].tolist())
print(cer)